In [2]:
#import haversine as hs
import json
import numpy as np
import os 
import re
import pandas as pd
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import sys
from math import sin, cos, sqrt, atan2, radians
import os.path
import statistics
import math
from scipy.interpolate import interp1d
from matplotlib import rcParams
#import plotly.express as px

In [3]:
demo = pd.read_csv ('D:\Alon\Tele\Projects\DEMOGRAPHIC\Demographic & Technology.csv')
list(demo)
demo.rename(columns = {'קוד אישי':'Id',}, inplace = True)
#display(demo)

calc_order= pd.read_csv ('D:/Alon/Tele/Projects/DEMOGRAPHIC/calc_participant_participation_order.csv')
display(calc_order)
df1_merge = calc_order.merge(demo, how = 'left', on= ['Id'])
display(df1_merge)


,Unnamed: 0,Id,Scenario,Condition,triggered_by,WorldTime,WithinScenarioOrder,WithinIDOrder
0,1078,A1_012594,Training,NaN,Egocar,1900-01-01 10:01:33,1,1
1,0,A1_012594,ACC,LOAD1_TTC1,Egocar,1900-01-01 10:13:52,1,2
2,64,A1_012594,ACC,LOAD1_TTC2,Egocar,1900-01-01 10:13:52,2,3
3,203,A1_012594,ACC,LOAD2_TTC2,Egocar,1900-01-01 10:18:35,3,4
4,270,A1_012594,ACC,LOAD3_TTC1,Egocar,1900-01-01 10:23:23,4,5
...,...,...,...,...,...,...,...,...
1419,131295,A6_311212,Slalom,Slalom1_far,RealVehicle,1900-01-01 12:47:20,2,2
1420,131296,A6_311212,Slalom,Slalom1_far,Egocar,1900-01-01 12:51:54,3,3
1421,131502,A6_311212,Slalom,Slalom2_close,Egocar,1900-01-01 12:56:18,4,4
1422,131731,A6_341995,Slalom,Slalom1_close,Egocar,1900-01-01 15:38:47,1,1


,Unnamed: 0,Id,Scenario,Condition,triggered_by,WorldTime,WithinScenarioOrder,WithinIDOrder,Timestamp,גיל,...,"* האם יש ניסיון בסימולטור נהיגה? אם כן, פרט",מספר שנים שיש בבעלותך רישיון נהיגה,"האם יש נסיון המשחקי מחשב כגון need for speed * וכו? אם כן, פרט",האם היית מעורב/ת בתאונה בעקבות אי שמירת מרחק (את/ה הנהג /ת הפוגע/ת) סמן את תשובתך?,במידה וסימנת כן בסעיף הקודם נא רשום את מספר הפעמים שהמקרה הזה קרה,טכנולוגיות חדשות הופכות את החיים שלי ליותר קלים,אני יכול/ה לפענח ולהבין מוצרים ושירותים טכנולוגיים לבד ללא עזרת אחרים,אני נהנה/נהנית לפענח ולהבין איך משתמשים בטכנולוגיה חדשה,"* האם יש ניסיון בסימולטור נהיגה? אם כן, פרט.1",אני נהנה/נהנית לפענח ולהבין איך משתמשים בטכנולוגיה חדשה.1
0,1078,A1_012594,Training,NaN,Egocar,1900-01-01 10:01:33,1,1,6/3/2021 9:54:42,24.0,...,לא,7.0,לא,לא,NaN,5.0,4.0,4.0,לא,NaN
1,0,A1_012594,ACC,LOAD1_TTC1,Egocar,1900-01-01 10:13:52,1,2,6/3/2021 9:54:42,24.0,...,לא,7.0,לא,לא,NaN,5.0,4.0,4.0,לא,NaN
2,64,A1_012594,ACC,LOAD1_TTC2,Egocar,1900-01-01 10:13:52,2,3,6/3/2021 9:54:42,24.0,...,לא,7.0,לא,לא,NaN,5.0,4.0,4.0,לא,NaN
3,203,A1_012594,ACC,LOAD2_TTC2,Egocar,1900-01-01 10:18:35,3,4,6/3/2021 9:54:42,24.0,...,לא,7.0,לא,לא,NaN,5.0,4.0,4.0,לא,NaN
4,270,A1_012594,ACC,LOAD3_TTC1,Egocar,1900-01-01 10:23:23,4,5,6/3/2021 9:54:42,24.0,...,לא,7.0,לא,לא,NaN,5.0,4.0,4.0,לא,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1643,131295,A6_311212,Slalom,Slalom1_far,RealVehicle,1900-01-01 12:47:20,2,2,10/10/2021 10:08:13,23.0,...,NaN,5.0,לא,לא,NaN,5.0,5.0,5.0,לא,NaN
1644,131296,A6_311212,Slalom,Slalom1_far,Egocar,1900-01-01 12:51:54,3,3,10/10/2021 10:08:13,23.0,...,NaN,5.0,לא,לא,NaN,5.0,5.0,5.0,לא,NaN
1645,131502,A6_311212,Slalom,Slalom2_close,Egocar,1900-01-01 12:56:18,4,4,10/10/2021 10:08:13,23.0,...,NaN,5.0,לא,לא,NaN,5.0,5.0,5.0,לא,NaN
1646,131731,A6_341995,Slalom,Slalom1_close,Egocar,1900-01-01 15:38:47,1,1,11/28/2021 0:17:57,27.0,...,NaN,6.0,לא,כן,2,5.0,4.0,5.0,לא,NaN


In [4]:
df1_merge.rename(columns = {'גיל':'Age','מגדר': 'Gender',
                           '* האם יש ניסיון בסימולטור נהיגה? אם כן, פרט':'experience simulator',
                           'מספר שנים שיש בבעלותך רישיון נהיגה':'drivers license years',
                           'האם יש נסיון המשחקי מחשב כגון need for speed * וכו? אם כן, פרט':'experience computer games',
                           'האם היית מעורב/ת בתאונה בעקבות אי שמירת מרחק (את/ה הנהג /ת הפוגע/ת) סמן את תשובתך?':'accident non-keeping distance',
                            'במידה וסימנת כן בסעיף הקודם נא רשום את מספר הפעמים שהמקרה הזה קרה': 'number of times',
                            'טכנולוגיות חדשות הופכות את החיים שלי ליותר קלים': 'technologies life easier',
                           'אני יכול/ה לפענח ולהבין מוצרים ושירותים טכנולוגיים לבד ללא עזרת אחרים':'understand technology',
                           'אני נהנה/נהנית לפענח ולהבין איך משתמשים בטכנולוגיה חדשה': 'enjoy understanding technology',
                            'במידה וסימנת כן בסעיף הקודם  נא רשום את מספר הפעמים שהמקרה הזה קרה ':'num of times',
                           'שנת לימודים':'status'  },    inplace = True)
df1_merge['Condition'] = df1_merge['Condition'].replace(["Slalom1_close"],"1")
df1_merge['Condition'] = df1_merge['Condition'].replace(["Slalom2_close"],"2")
df1_merge['Condition'] = df1_merge['Condition'].replace(["Slalom1_far"],"1")
df1_merge['Condition'] = df1_merge['Condition'].replace(["Slalom2_far"],"2")
df1_merge['Gender'] = df1_merge['Gender'].replace(['Male'],1)
df1_merge['Gender'] = df1_merge['Gender'].replace(['Female'],0)
df1_merge['experience simulator'] = df1_merge['experience simulator'] .fillna('לא, אין ניסיון')
df1_merge['num of times'] = df1_merge['num of times'] .fillna(0)
df1_merge=df1_merge.drop(['* האם יש ניסיון בסימולטור נהיגה? אם כן, פרט.1','אני נהנה/נהנית לפענח ולהבין איך משתמשים בטכנולוגיה חדשה.1'], axis =1,)
display(df1_merge)
print(df1_merge['Condition'].unique())



,Unnamed: 0,Id,Scenario,Condition,triggered_by,WorldTime,WithinScenarioOrder,WithinIDOrder,Timestamp,Age,Gender,status,experience simulator,drivers license years,experience computer games,accident non-keeping distance,num of times,technologies life easier,understand technology,enjoy understanding technology
0,1078,A1_012594,Training,NaN,Egocar,1900-01-01 10:01:33,1,1,6/3/2021 9:54:42,24.0,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
1,0,A1_012594,ACC,LOAD1_TTC1,Egocar,1900-01-01 10:13:52,1,2,6/3/2021 9:54:42,24.0,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
2,64,A1_012594,ACC,LOAD1_TTC2,Egocar,1900-01-01 10:13:52,2,3,6/3/2021 9:54:42,24.0,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
3,203,A1_012594,ACC,LOAD2_TTC2,Egocar,1900-01-01 10:18:35,3,4,6/3/2021 9:54:42,24.0,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
4,270,A1_012594,ACC,LOAD3_TTC1,Egocar,1900-01-01 10:23:23,4,5,6/3/2021 9:54:42,24.0,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1643,131295,A6_311212,Slalom,1,RealVehicle,1900-01-01 12:47:20,2,2,10/10/2021 10:08:13,23.0,נקבה,תואר ראשון - שנה שלישית,"לא, אין ניסיון",5.0,לא,לא,0,5.0,5.0,5.0
1644,131296,A6_311212,Slalom,1,Egocar,1900-01-01 12:51:54,3,3,10/10/2021 10:08:13,23.0,נקבה,תואר ראשון - שנה שלישית,"לא, אין ניסיון",5.0,לא,לא,0,5.0,5.0,5.0
1645,131502,A6_311212,Slalom,2,Egocar,1900-01-01 12:56:18,4,4,10/10/2021 10:08:13,23.0,נקבה,תואר ראשון - שנה שלישית,"לא, אין ניסיון",5.0,לא,לא,0,5.0,5.0,5.0
1646,131731,A6_341995,Slalom,1,Egocar,1900-01-01 15:38:47,1,1,11/28/2021 0:17:57,27.0,נקבה,תואר ראשון - שנה שלישית,"לא, אין ניסיון",6.0,לא,כן,2,5.0,4.0,5.0


[nan 'LOAD1_TTC1' 'LOAD1_TTC2' 'LOAD2_TTC2' 'LOAD3_TTC1' 'LOAD2_TTC1'
 'LOAD3_TTC2' 'Latency1' 'Latency2' 'Latency3' 'Egocentric' 'Allocentric'
 '2' '1']


In [5]:
df1_merge.drop(['Unnamed: 0'], axis =1)
df1_merge.drop(['Timestamp'], axis =1)

,Unnamed: 0,Id,Scenario,Condition,triggered_by,WorldTime,WithinScenarioOrder,WithinIDOrder,Age,Gender,status,experience simulator,drivers license years,experience computer games,accident non-keeping distance,num of times,technologies life easier,understand technology,enjoy understanding technology
0,1078,A1_012594,Training,NaN,Egocar,1900-01-01 10:01:33,1,1,24.0,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
1,0,A1_012594,ACC,LOAD1_TTC1,Egocar,1900-01-01 10:13:52,1,2,24.0,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
2,64,A1_012594,ACC,LOAD1_TTC2,Egocar,1900-01-01 10:13:52,2,3,24.0,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
3,203,A1_012594,ACC,LOAD2_TTC2,Egocar,1900-01-01 10:18:35,3,4,24.0,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
4,270,A1_012594,ACC,LOAD3_TTC1,Egocar,1900-01-01 10:23:23,4,5,24.0,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1643,131295,A6_311212,Slalom,1,RealVehicle,1900-01-01 12:47:20,2,2,23.0,נקבה,תואר ראשון - שנה שלישית,"לא, אין ניסיון",5.0,לא,לא,0,5.0,5.0,5.0
1644,131296,A6_311212,Slalom,1,Egocar,1900-01-01 12:51:54,3,3,23.0,נקבה,תואר ראשון - שנה שלישית,"לא, אין ניסיון",5.0,לא,לא,0,5.0,5.0,5.0
1645,131502,A6_311212,Slalom,2,Egocar,1900-01-01 12:56:18,4,4,23.0,נקבה,תואר ראשון - שנה שלישית,"לא, אין ניסיון",5.0,לא,לא,0,5.0,5.0,5.0
1646,131731,A6_341995,Slalom,1,Egocar,1900-01-01 15:38:47,1,1,27.0,נקבה,תואר ראשון - שנה שלישית,"לא, אין ניסיון",6.0,לא,כן,2,5.0,4.0,5.0


In [6]:
print(df1_merge['Scenario'].unique())

['Training' 'ACC' 'Latency' 'Spatial' 'Slalom']


In [7]:
df_gropued= df1_merge.groupby(["Scenario",'Condition', 'triggered_by']).agg(
    min_Age=pd.NamedAgg(column="Age", aggfunc="min"),
    max_Age=pd.NamedAgg(column="Age", aggfunc="max"),
    average_Age=pd.NamedAgg(column="Age", aggfunc=np.mean),
    Std_Age=pd.NamedAgg(column="Age", aggfunc=np.std),
    average_license=pd.NamedAgg(column="drivers license years", aggfunc=np.mean),
    average_technologies_life_easier=pd.NamedAgg(column="technologies life easier", aggfunc=np.mean),
    average_understand_technologies=pd.NamedAgg(column="understand technology", aggfunc=np.mean),
    average_enjoy_understanding_technologies=pd.NamedAgg(column="enjoy understanding technology", aggfunc=np.mean),  
  
)
display(df_gropued)

min_Age  max_Age  average_Age   Std_Age  \
Scenario Condition   triggered_by                                            
ACC      LOAD1_TTC1  Egocar           19.0     30.0    24.604724  1.767873   
         LOAD1_TTC2  Egocar           19.0     30.0    24.538053  1.720482   
         LOAD2_TTC1  Egocar           19.0     28.0    24.524444  1.690865   
         LOAD2_TTC2  Egocar           19.0     28.0    24.755118  1.139323   
         LOAD3_TTC1  Egocar           19.0     32.0    24.507273  1.834767   
         LOAD3_TTC2  Egocar           19.0     32.0    24.498214  1.809484   
Latency  Latency1    Egocar           19.0     30.0    24.502970  1.834909   
         Latency2    Egocar           19.0     32.0    24.512871  1.888580   
         Latency3    Egocar           19.0     28.0    24.468627  1.742991   
Slalom   1           Egocar           23.0     32.0    25.873171  1.749146   
                     RealVehicle      23.0     32.0    25.866667  1.779612   
         2           Egocar           23.0     32.0    25.859459  1.766267   
                     RealVehicle      23.0     32.0    25.845000  1.761985   
Spatial  Allocentric Egocar           19.0     32.0    24.518000  1.892056   
         Egocentric  Egocar           19.0     30.0    24.497872  1.786868   

                                   average_license  \
Scenario Condition   triggered_by                    
ACC      LOAD1_TTC1  Egocar               6.539370   
         LOAD1_TTC2  Egocar               6.491150   
         LOAD2_TTC1  Egocar               6.381481   
         LOAD2_TTC2  Egocar               6.194882   
         LOAD3_TTC1  Egocar               6.513636   
         LOAD3_TTC2  Egocar               6.325893   
Latency  Latency1    Egocar               6.529703   
         Latency2    Egocar               6.519802   
         Latency3    Egocar               6.387255   
Slalom   1           Egocar               7.731707   
                     RealVehicle          7.794872   
         2           Egocar               7.756757   
                     RealVehicle          7.775000   
Spatial  Allocentric Egocar               6.415000   
         Egocentric  Egocar               6.553191   

                                   average_technologies_life_easier  \
Scenario Condition   triggered_by                                     
ACC      LOAD1_TTC1  Egocar                                4.551181   
         LOAD1_TTC2  Egocar                                4.424779   
         LOAD2_TTC1  Egocar                                4.348148   
         LOAD2_TTC2  Egocar                                3.622047   
         LOAD3_TTC1  Egocar                                4.463636   
         LOAD3_TTC2  Egocar                                4.428571   
Latency  Latency1    Egocar                                4.504950   
         Latency2    Egocar                                4.504950   
         Latency3    Egocar                                4.558824   
Slalom   1           Egocar                                4.658537   
                     RealVehicle                           4.666667   
         2           Egocar                                4.621622   
                     RealVehicle                           4.650000   
Spatial  Allocentric Egocar                                4.500000   
         Egocentric  Egocar                                4.521277   

                                   average_understand_technologies  \
Scenario Condition   triggered_by                                    
ACC      LOAD1_TTC1  Egocar                               3.803150   
         LOAD1_TTC2  Egocar                               3.805310   
         LOAD2_TTC1  Egocar                               3.837037   
         LOAD2_TTC2  Egocar                               3.326772   
         LOAD3_TTC1  Egocar                               3.845455   
         LOAD3_TTC2  Egocar                               3.794643   
Latency  Latency1    

In [8]:
df1_merge= df1_merge.drop(df1_merge.index[df1_merge['Scenario']=='Training'], inplace=False )
display(df1_merge)

,Unnamed: 0,Id,Scenario,Condition,triggered_by,WorldTime,WithinScenarioOrder,WithinIDOrder,Timestamp,Age,Gender,status,experience simulator,drivers license years,experience computer games,accident non-keeping distance,num of times,technologies life easier,understand technology,enjoy understanding technology
1,0,A1_012594,ACC,LOAD1_TTC1,Egocar,1900-01-01 10:13:52,1,2,6/3/2021 9:54:42,24.0,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
2,64,A1_012594,ACC,LOAD1_TTC2,Egocar,1900-01-01 10:13:52,2,3,6/3/2021 9:54:42,24.0,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
3,203,A1_012594,ACC,LOAD2_TTC2,Egocar,1900-01-01 10:18:35,3,4,6/3/2021 9:54:42,24.0,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
4,270,A1_012594,ACC,LOAD3_TTC1,Egocar,1900-01-01 10:23:23,4,5,6/3/2021 9:54:42,24.0,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
5,128,A1_012594,ACC,LOAD2_TTC1,Egocar,1900-01-01 10:30:34,5,6,6/3/2021 9:54:42,24.0,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1642,131501,A6_311212,Slalom,2,RealVehicle,1900-01-01 12:41:51,1,1,10/10/2021 10:08:13,23.0,נקבה,תואר ראשון - שנה שלישית,"לא, אין ניסיון",5.0,לא,לא,0,5.0,5.0,5.0
1643,131295,A6_311212,Slalom,1,RealVehicle,1900-01-01 12:47:20,2,2,10/10/2021 10:08:13,23.0,נקבה,תואר ראשון - שנה שלישית,"לא, אין ניסיון",5.0,לא,לא,0,5.0,5.0,5.0
1644,131296,A6_311212,Slalom,1,Egocar,1900-01-01 12:51:54,3,3,10/10/2021 10:08:13,23.0,נקבה,תואר ראשון - שנה שלישית,"לא, אין ניסיון",5.0,לא,לא,0,5.0,5.0,5.0
1645,131502,A6_311212,Slalom,2,Egocar,1900-01-01 12:56:18,4,4,10/10/2021 10:08:13,23.0,נקבה,תואר ראשון - שנה שלישית,"לא, אין ניסיון",5.0,לא,לא,0,5.0,5.0,5.0


In [5]:
demo_234= pd.read_csv('D:\\Alon\\Tele\\Projects\\DEMOGRAPHIC\\NASA_TLX_after_each_session.csv')

demo_234.groupby(['Id','Scenario']).agg(    
    average_require_mentally=pd.NamedAgg(column="require mentally", aggfunc=np.mean),
    average_require_physically=pd.NamedAgg(column="require physically", aggfunc=np.mean),
    average_required_execution_rate=pd.NamedAgg(column="required execution rate:", aggfunc=np.mean),
    average_successful_performing=pd.NamedAgg(column="successful performing", aggfunc=np.mean),
    average_work_hard=pd.NamedAgg(column="required to work hard", aggfunc=np.mean),
    average_Frustration_level=pd.NamedAgg(column="Frustration level", aggfunc=np.mean),
                                                        )
                                                        

display(demo_234)

,Timestamp,Id,Scenario,require mentally,require physically,required execution rate:,successful performing,required to work hard,Frustration level
0,3/29/2021 16:30:41,A3_002019,Latency 100 MS,5,5.0,5.0,9.0,5.0,8.0
1,3/29/2021 16:44:48,A3_002019,Latency 300 MS,5,5.0,5.0,9.0,5.0,8.0
2,3/29/2021 16:59:21,A3_002019,ACC TTC 1 ROAD LOAD LEVEL 1,7,5.0,7.0,5.0,5.0,7.0
3,3/29/2021 17:08:15,A3_002019,ACC TTC 2 ROAD LOAD LEVEL 1,5,5.0,5.0,5.0,5.0,4.0
4,3/29/2021 17:22:36,A3_002019,Spatial orientation Egocentric (cameras inside...,5,5.0,5.0,5.0,5.0,5.0
...,...,...,...,...,...,...,...,...,...
1011,8/30/2021 11:28:54,A2_289779,Latency1-tele-far,5,5.0,5.0,7.0,5.0,3.0
1012,8/30/2021 11:35:26,A2_289779,Latency1-sim-close,4,5.0,5.0,9.0,4.0,4.0
1013,8/30/2021 11:43:22,A2_289779,Latency2-sim-far,4,5.0,6.0,6.0,6.0,6.0
1014,8/30/2021 12:31:16,A6_299929,Latency1-sim-close,4,4.0,NaN,8.0,3.0,4.0


In [10]:
demo_234.drop(['Timestamp'], axis =1)


,Id,Scenario,require mentally,require physically,required execution rate:,successful performing,required to work hard,Frustration level
0,A3_002019,Latency 100 MS,5,5.0,5.0,9.0,5.0,8.0
1,A3_002019,Latency 300 MS,5,5.0,5.0,9.0,5.0,8.0
2,A3_002019,ACC TTC 1 ROAD LOAD LEVEL 1,7,5.0,7.0,5.0,5.0,7.0
3,A3_002019,ACC TTC 2 ROAD LOAD LEVEL 1,5,5.0,5.0,5.0,5.0,4.0
4,A3_002019,Spatial orientation Egocentric (cameras inside...,5,5.0,5.0,5.0,5.0,5.0
...,...,...,...,...,...,...,...,...
1011,A2_289779,Latency1-tele-far,5,5.0,5.0,7.0,5.0,3.0
1012,A2_289779,Latency1-sim-close,4,5.0,5.0,9.0,4.0,4.0
1013,A2_289779,Latency2-sim-far,4,5.0,6.0,6.0,6.0,6.0
1014,A6_299929,Latency1-sim-close,4,4.0,NaN,8.0,3.0,4.0


In [11]:
demo_234.loc[demo_234['Scenario']=='Latency 0 MS', 'Condition'] = 'Latency1'
demo_234.loc[demo_234['Scenario']=='Latency 100 MS', 'Condition'] = 'Latency1'
demo_234.loc[demo_234['Scenario']=='Latency 150 MS', 'Condition'] = 'Latency2'
demo_234.loc[demo_234['Scenario']=='Latency 200 MS', 'Condition'] = 'Latency2'
demo_234.loc[demo_234['Scenario']=='Latency 250 MS', 'Condition'] = 'Latency3'
demo_234.loc[demo_234['Scenario']=='Latency 300 MS', 'Condition'] = 'Latency3'
demo_234.loc[demo_234['Scenario']=='ACC TTC 1 ROAD LOAD LEVEL 1', 'Condition'] = 'LOAD1_TTC1'
demo_234.loc[demo_234['Scenario']=='ACC TTC 1 ROAD LOAD LEVEL 2', 'Condition'] = 'LOAD1_TTC2'
demo_234.loc[demo_234['Scenario']=='ACC TTC 2 ROAD LOAD LEVEL 1', 'Condition'] = 'LOAD2_TTC1'
demo_234.loc[demo_234['Scenario']=='ACC TTC 2 ROAD LOAD LEVEL 2', 'Condition'] = 'LOAD2_TTC2'
demo_234.loc[demo_234['Scenario']=='ACC TTC 1 ROAD LOAD LEVEL 3', 'Condition'] = 'LOAD3_TTC1'
demo_234.loc[demo_234['Scenario']=='ACC TTC 2 ROAD LOAD LEVEL 3', 'Condition'] = 'LOAD3_TTC2'
demo_234.loc[demo_234['Scenario']=='Spatial orientation Allocentric (cameras on the roof of the car)', 'Condition'] = 'Allocentric'
demo_234.loc[demo_234['Scenario']=='Spatial orientation Egocentric (cameras inside the car)', 'Condition'] = 'Egocentric'
demo_234.loc[demo_234['Scenario']=='Latency1-tele-far', 'Condition'] = '1'
demo_234.loc[demo_234['Scenario']=='Latency1-tele-close', 'Condition'] = '1'
demo_234.loc[demo_234['Scenario']=='Latency2-tele-far', 'Condition'] = '2'
demo_234.loc[demo_234['Scenario']=='Latency2-tele-close', 'Condition'] = '2'
demo_234.loc[demo_234['Scenario']=='Latency1-sim-far', 'Condition'] = '1'
demo_234.loc[demo_234['Scenario']=='Latency1-sim-close', 'Condition'] = '1'
demo_234.loc[demo_234['Scenario']=='Latency2-sim-far', 'Condition'] = '2'
demo_234.loc[demo_234['Scenario']=='Latency2-sim-close', 'Condition'] = '2'


display(demo_234)

,Timestamp,Id,Scenario,require mentally,require physically,required execution rate:,successful performing,required to work hard,Frustration level,Condition
0,3/29/2021 16:30:41,A3_002019,Latency 100 MS,5,5.0,5.0,9.0,5.0,8.0,Latency1
1,3/29/2021 16:44:48,A3_002019,Latency 300 MS,5,5.0,5.0,9.0,5.0,8.0,Latency3
2,3/29/2021 16:59:21,A3_002019,ACC TTC 1 ROAD LOAD LEVEL 1,7,5.0,7.0,5.0,5.0,7.0,LOAD1_TTC1
3,3/29/2021 17:08:15,A3_002019,ACC TTC 2 ROAD LOAD LEVEL 1,5,5.0,5.0,5.0,5.0,4.0,LOAD2_TTC1
4,3/29/2021 17:22:36,A3_002019,Spatial orientation Egocentric (cameras inside...,5,5.0,5.0,5.0,5.0,5.0,Egocentric
...,...,...,...,...,...,...,...,...,...,...
1011,8/30/2021 11:28:54,A2_289779,Latency1-tele-far,5,5.0,5.0,7.0,5.0,3.0,1
1012,8/30/2021 11:35:26,A2_289779,Latency1-sim-close,4,5.0,5.0,9.0,4.0,4.0,1
1013,8/30/2021 11:43:22,A2_289779,Latency2-sim-far,4,5.0,6.0,6.0,6.0,6.0,2
1014,8/30/2021 12:31:16,A6_299929,Latency1-sim-close,4,4.0,NaN,8.0,3.0,4.0,1


In [12]:
demo_234['triggered_by'] = 'Egocar'
demo_234.loc[demo_234['Scenario']=='Latency1-tele-far', 'triggered_by'] = 'RealVehicle'
demo_234.loc[demo_234['Scenario']=='Latency2-tele-far', 'triggered_by'] = 'RealVehicle'
demo_234.loc[demo_234['Scenario']=='Latency1-tele-close', 'triggered_by'] = 'RealVehicle'
demo_234.loc[demo_234['Scenario']=='Latency2-tele-close', 'triggered_by'] = 'RealVehicle'

display(demo_234)

,Timestamp,Id,Scenario,require mentally,require physically,required execution rate:,successful performing,required to work hard,Frustration level,Condition,triggered_by
0,3/29/2021 16:30:41,A3_002019,Latency 100 MS,5,5.0,5.0,9.0,5.0,8.0,Latency1,Egocar
1,3/29/2021 16:44:48,A3_002019,Latency 300 MS,5,5.0,5.0,9.0,5.0,8.0,Latency3,Egocar
2,3/29/2021 16:59:21,A3_002019,ACC TTC 1 ROAD LOAD LEVEL 1,7,5.0,7.0,5.0,5.0,7.0,LOAD1_TTC1,Egocar
3,3/29/2021 17:08:15,A3_002019,ACC TTC 2 ROAD LOAD LEVEL 1,5,5.0,5.0,5.0,5.0,4.0,LOAD2_TTC1,Egocar
4,3/29/2021 17:22:36,A3_002019,Spatial orientation Egocentric (cameras inside...,5,5.0,5.0,5.0,5.0,5.0,Egocentric,Egocar
...,...,...,...,...,...,...,...,...,...,...,...
1011,8/30/2021 11:28:54,A2_289779,Latency1-tele-far,5,5.0,5.0,7.0,5.0,3.0,1,RealVehicle
1012,8/30/2021 11:35:26,A2_289779,Latency1-sim-close,4,5.0,5.0,9.0,4.0,4.0,1,Egocar
1013,8/30/2021 11:43:22,A2_289779,Latency2-sim-far,4,5.0,6.0,6.0,6.0,6.0,2,Egocar
1014,8/30/2021 12:31:16,A6_299929,Latency1-sim-close,4,4.0,NaN,8.0,3.0,4.0,1,Egocar


In [13]:
demo_234 = demo_234.replace({'Scenario':'Latency1-sim-close'},'Slalom')
demo_234 = demo_234.replace({'Scenario':'Latency2-sim-far'},'Slalom')
demo_234 = demo_234.replace({'Scenario':'Latency2-tele-far'},'Slalom')
demo_234 = demo_234.replace({'Scenario':'Latency2-tele-close'},'Slalom')
demo_234 = demo_234.replace({'Scenario':'Latency1-tele-close'},'Slalom')
demo_234 = demo_234.replace({'Scenario':'Latency1-sim-far'},'Slalom')
demo_234 = demo_234.replace({'Scenario':'Latency2-sim-close'},'Slalom')
demo_234 = demo_234.replace({'Scenario':'Latency1-tele-far'},'Slalom')
display(demo_234)

print(demo_234['Scenario'].unique())


,Timestamp,Id,Scenario,require mentally,require physically,required execution rate:,successful performing,required to work hard,Frustration level,Condition,triggered_by
0,3/29/2021 16:30:41,A3_002019,Latency 100 MS,5,5.0,5.0,9.0,5.0,8.0,Latency1,Egocar
1,3/29/2021 16:44:48,A3_002019,Latency 300 MS,5,5.0,5.0,9.0,5.0,8.0,Latency3,Egocar
2,3/29/2021 16:59:21,A3_002019,ACC TTC 1 ROAD LOAD LEVEL 1,7,5.0,7.0,5.0,5.0,7.0,LOAD1_TTC1,Egocar
3,3/29/2021 17:08:15,A3_002019,ACC TTC 2 ROAD LOAD LEVEL 1,5,5.0,5.0,5.0,5.0,4.0,LOAD2_TTC1,Egocar
4,3/29/2021 17:22:36,A3_002019,Spatial orientation Egocentric (cameras inside...,5,5.0,5.0,5.0,5.0,5.0,Egocentric,Egocar
...,...,...,...,...,...,...,...,...,...,...,...
1011,8/30/2021 11:28:54,A2_289779,Slalom,5,5.0,5.0,7.0,5.0,3.0,1,RealVehicle
1012,8/30/2021 11:35:26,A2_289779,Slalom,4,5.0,5.0,9.0,4.0,4.0,1,Egocar
1013,8/30/2021 11:43:22,A2_289779,Slalom,4,5.0,6.0,6.0,6.0,6.0,2,Egocar
1014,8/30/2021 12:31:16,A6_299929,Slalom,4,4.0,NaN,8.0,3.0,4.0,1,Egocar


['Latency 100 MS' 'Latency 300 MS' 'ACC TTC 1 ROAD LOAD LEVEL 1'
 'ACC TTC 2 ROAD LOAD LEVEL 1'
 'Spatial orientation Egocentric (cameras inside the car)'
 'ACC TTC 2 ROAD LOAD LEVEL 2' 'ACC TTC 1 ROAD LOAD LEVEL 3'
 'ACC TTC 1 ROAD LOAD LEVEL 2' 'ACC TTC 2 ROAD LOAD LEVEL 3'
 'Latency 200 MS'
 'Spatial orientation Allocentric (cameras on the roof of the car)'
 'Latency 0 MS' 'Latency 250 MS' 'Latency 150 MS' 'Slalom']


In [14]:
demo_234 = demo_234.replace({'Scenario':'Spatial orientation Allocentric (cameras on the roof of the car)'},'Spatial')
demo_234 = demo_234.replace({'Scenario':'Spatial orientation Egocentric (cameras inside the car)'},'Spatial')
print(demo_234['Scenario'].unique())

['Latency 100 MS' 'Latency 300 MS' 'ACC TTC 1 ROAD LOAD LEVEL 1'
 'ACC TTC 2 ROAD LOAD LEVEL 1' 'Spatial' 'ACC TTC 2 ROAD LOAD LEVEL 2'
 'ACC TTC 1 ROAD LOAD LEVEL 3' 'ACC TTC 1 ROAD LOAD LEVEL 2'
 'ACC TTC 2 ROAD LOAD LEVEL 3' 'Latency 200 MS' 'Latency 0 MS'
 'Latency 250 MS' 'Latency 150 MS' 'Slalom']


In [15]:
demo_234 = demo_234.replace({'Scenario':'ACC TTC 1 ROAD LOAD LEVEL 1'},'ACC')
demo_234 = demo_234.replace({'Scenario':'ACC TTC 1 ROAD LOAD LEVEL 2'},'ACC')
demo_234 = demo_234.replace({'Scenario':'ACC TTC 1 ROAD LOAD LEVEL 3'},'ACC')
demo_234 = demo_234.replace({'Scenario':'ACC TTC 2 ROAD LOAD LEVEL 1'},'ACC')
demo_234 = demo_234.replace({'Scenario':'ACC TTC 2 ROAD LOAD LEVEL 2'},'ACC')
demo_234 = demo_234.replace({'Scenario':'ACC TTC 2 ROAD LOAD LEVEL 3'},'ACC')
print(demo_234['Scenario'].unique())

['Latency 100 MS' 'Latency 300 MS' 'ACC' 'Spatial' 'Latency 200 MS'
 'Latency 0 MS' 'Latency 250 MS' 'Latency 150 MS' 'Slalom']


In [16]:
demo_234 = demo_234.replace({'Scenario':'Latency 0 MS'},'Latency')
demo_234 = demo_234.replace({'Scenario':'Latency 100 MS'},'Latency')
demo_234 = demo_234.replace({'Scenario':'Latency 150 MS'},'Latency')
demo_234 = demo_234.replace({'Scenario':'Latency 200 MS'},'Latency')
demo_234 = demo_234.replace({'Scenario':'Latency 250 MS'},'Latency')
demo_234 = demo_234.replace({'Scenario':'Latency 300 MS'},'Latency')
print(demo_234['Scenario'].unique())

['Latency' 'ACC' 'Spatial' 'Slalom']


In [17]:
print(df1_merge['Condition'].unique())
print(demo_234['Condition'].unique())

['LOAD1_TTC1' 'LOAD1_TTC2' 'LOAD2_TTC2' 'LOAD3_TTC1' 'LOAD2_TTC1'
 'LOAD3_TTC2' 'Latency1' 'Latency2' 'Latency3' 'Egocentric' 'Allocentric'
 '2' '1']
['Latency1' 'Latency3' 'LOAD1_TTC1' 'LOAD2_TTC1' 'Egocentric' 'LOAD2_TTC2'
 'LOAD3_TTC1' 'LOAD1_TTC2' 'LOAD3_TTC2' 'Latency2' 'Allocentric' '1' '2']


In [18]:
print(df1_merge['Scenario'].unique())
print(demo_234['Scenario'].unique())

['ACC' 'Latency' 'Spatial' 'Slalom']
['Latency' 'ACC' 'Spatial' 'Slalom']


In [19]:
display(demo_234)
demo_234.duplicated().sum()

,Timestamp,Id,Scenario,require mentally,require physically,required execution rate:,successful performing,required to work hard,Frustration level,Condition,triggered_by
0,3/29/2021 16:30:41,A3_002019,Latency,5,5.0,5.0,9.0,5.0,8.0,Latency1,Egocar
1,3/29/2021 16:44:48,A3_002019,Latency,5,5.0,5.0,9.0,5.0,8.0,Latency3,Egocar
2,3/29/2021 16:59:21,A3_002019,ACC,7,5.0,7.0,5.0,5.0,7.0,LOAD1_TTC1,Egocar
3,3/29/2021 17:08:15,A3_002019,ACC,5,5.0,5.0,5.0,5.0,4.0,LOAD2_TTC1,Egocar
4,3/29/2021 17:22:36,A3_002019,Spatial,5,5.0,5.0,5.0,5.0,5.0,Egocentric,Egocar
...,...,...,...,...,...,...,...,...,...,...,...
1011,8/30/2021 11:28:54,A2_289779,Slalom,5,5.0,5.0,7.0,5.0,3.0,1,RealVehicle
1012,8/30/2021 11:35:26,A2_289779,Slalom,4,5.0,5.0,9.0,4.0,4.0,1,Egocar
1013,8/30/2021 11:43:22,A2_289779,Slalom,4,5.0,6.0,6.0,6.0,6.0,2,Egocar
1014,8/30/2021 12:31:16,A6_299929,Slalom,4,4.0,NaN,8.0,3.0,4.0,1,Egocar


0

In [20]:
demo_234.drop(['Timestamp'], axis =1)

,Id,Scenario,require mentally,require physically,required execution rate:,successful performing,required to work hard,Frustration level,Condition,triggered_by
0,A3_002019,Latency,5,5.0,5.0,9.0,5.0,8.0,Latency1,Egocar
1,A3_002019,Latency,5,5.0,5.0,9.0,5.0,8.0,Latency3,Egocar
2,A3_002019,ACC,7,5.0,7.0,5.0,5.0,7.0,LOAD1_TTC1,Egocar
3,A3_002019,ACC,5,5.0,5.0,5.0,5.0,4.0,LOAD2_TTC1,Egocar
4,A3_002019,Spatial,5,5.0,5.0,5.0,5.0,5.0,Egocentric,Egocar
...,...,...,...,...,...,...,...,...,...,...
1011,A2_289779,Slalom,5,5.0,5.0,7.0,5.0,3.0,1,RealVehicle
1012,A2_289779,Slalom,4,5.0,5.0,9.0,4.0,4.0,1,Egocar
1013,A2_289779,Slalom,4,5.0,6.0,6.0,6.0,6.0,2,Egocar
1014,A6_299929,Slalom,4,4.0,NaN,8.0,3.0,4.0,1,Egocar


In [21]:
df2_merge = demo_234.merge(df1_merge, how = 'inner', on= ['Id','Scenario', 'Condition','triggered_by'])
display(df2_merge)
df2_merge.drop(['Timestamp_x', 'Timestamp_y','WorldTime'], axis =1)

,Timestamp_x,Id,Scenario,require mentally,require physically,required execution rate:,successful performing,required to work hard,Frustration level,Condition,...,Gender,status,experience simulator,drivers license years,experience computer games,accident non-keeping distance,num of times,technologies life easier,understand technology,enjoy understanding technology
0,04/04/2021 10:12,A1_012594,ACC,7,2.0,7.0,6.0,2.0,1.0,LOAD2_TTC1,...,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
1,04/04/2021 10:17,A1_012594,ACC,6,2.0,5.0,8.0,2.0,1.0,LOAD1_TTC1,...,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
2,04/04/2021 10:21,A1_012594,ACC,7,2.0,8.0,1.0,2.0,1.0,LOAD2_TTC2,...,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
3,04/04/2021 10:29,A1_012594,ACC,7,3.0,6.0,1.0,5.0,3.0,LOAD3_TTC1,...,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
4,04/04/2021 10:34,A1_012594,ACC,8,2.0,6.0,9.0,4.0,2.0,LOAD1_TTC2,...,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1381,8/30/2021 11:08:21,A5_292076,Slalom,3,5.0,6.0,7.0,4.0,2.0,2,...,זכר,תואר ראשון - שנה שלישית,"לא, אין ניסיון",6.0,"כן, אבל בקטנה ממש",לא,0,5.0,4.0,4.0
1382,8/30/2021 11:23:52,A2_289779,Slalom,7,NaN,6.0,7.0,5.0,5.0,2,...,זכר,תואר ראשון - שנה רביעית,"לא, אין ניסיון",8.0,כן,כן,1,5.0,5.0,4.0
1383,8/30/2021 11:28:54,A2_289779,Slalom,5,5.0,5.0,7.0,5.0,3.0,1,...,זכר,תואר ראשון - שנה רביעית,"לא, אין ניסיון",8.0,כן,כן,1,5.0,5.0,4.0
1384,8/30/2021 11:35:26,A2_289779,Slalom,4,5.0,5.0,9.0,4.0,4.0,1,...,זכר,תואר ראשון - שנה רביעית,"לא, אין ניסיון",8.0,כן,כן,1,5.0,5.0,4.0


,Id,Scenario,require mentally,require physically,required execution rate:,successful performing,required to work hard,Frustration level,Condition,triggered_by,...,Gender,status,experience simulator,drivers license years,experience computer games,accident non-keeping distance,num of times,technologies life easier,understand technology,enjoy understanding technology
0,A1_012594,ACC,7,2.0,7.0,6.0,2.0,1.0,LOAD2_TTC1,Egocar,...,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
1,A1_012594,ACC,6,2.0,5.0,8.0,2.0,1.0,LOAD1_TTC1,Egocar,...,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
2,A1_012594,ACC,7,2.0,8.0,1.0,2.0,1.0,LOAD2_TTC2,Egocar,...,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
3,A1_012594,ACC,7,3.0,6.0,1.0,5.0,3.0,LOAD3_TTC1,Egocar,...,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
4,A1_012594,ACC,8,2.0,6.0,9.0,4.0,2.0,LOAD1_TTC2,Egocar,...,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1381,A5_292076,Slalom,3,5.0,6.0,7.0,4.0,2.0,2,RealVehicle,...,זכר,תואר ראשון - שנה שלישית,"לא, אין ניסיון",6.0,"כן, אבל בקטנה ממש",לא,0,5.0,4.0,4.0
1382,A2_289779,Slalom,7,NaN,6.0,7.0,5.0,5.0,2,RealVehicle,...,זכר,תואר ראשון - שנה רביעית,"לא, אין ניסיון",8.0,כן,כן,1,5.0,5.0,4.0
1383,A2_289779,Slalom,5,5.0,5.0,7.0,5.0,3.0,1,RealVehicle,...,זכר,תואר ראשון - שנה רביעית,"לא, אין ניסיון",8.0,כן,כן,1,5.0,5.0,4.0
1384,A2_289779,Slalom,4,5.0,5.0,9.0,4.0,4.0,1,Egocar,...,זכר,תואר ראשון - שנה רביעית,"לא, אין ניסיון",8.0,כן,כן,1,5.0,5.0,4.0


In [22]:
df2_merge= df2_merge.drop(['Timestamp_x', 'Timestamp_y','WorldTime','Unnamed: 0'], axis =1)

In [23]:
df2_merge= df2_merge.drop(['WithinScenarioOrder', 'WithinIDOrder'], axis =1)

In [24]:
df2_merge.shape

(1386, 21)

In [25]:
df2_merge.duplicated(subset=['Id','Scenario','Condition', 'triggered_by']).sum()

435

In [26]:
df2_merge=df2_merge.drop_duplicates(subset=['Id','Scenario','Condition','triggered_by'])


In [27]:
df2_merge.shape
df2_merge.reset_index()

,index,Id,Scenario,require mentally,require physically,required execution rate:,successful performing,required to work hard,Frustration level,Condition,...,Gender,status,experience simulator,drivers license years,experience computer games,accident non-keeping distance,num of times,technologies life easier,understand technology,enjoy understanding technology
0,0,A1_012594,ACC,7,2.0,7.0,6.0,2.0,1.0,LOAD2_TTC1,...,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
1,1,A1_012594,ACC,6,2.0,5.0,8.0,2.0,1.0,LOAD1_TTC1,...,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
2,2,A1_012594,ACC,7,2.0,8.0,1.0,2.0,1.0,LOAD2_TTC2,...,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
3,3,A1_012594,ACC,7,3.0,6.0,1.0,5.0,3.0,LOAD3_TTC1,...,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
4,4,A1_012594,ACC,8,2.0,6.0,9.0,4.0,2.0,LOAD1_TTC2,...,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946,1381,A5_292076,Slalom,3,5.0,6.0,7.0,4.0,2.0,2,...,זכר,תואר ראשון - שנה שלישית,"לא, אין ניסיון",6.0,"כן, אבל בקטנה ממש",לא,0,5.0,4.0,4.0
947,1382,A2_289779,Slalom,7,NaN,6.0,7.0,5.0,5.0,2,...,זכר,תואר ראשון - שנה רביעית,"לא, אין ניסיון",8.0,כן,כן,1,5.0,5.0,4.0
948,1383,A2_289779,Slalom,5,5.0,5.0,7.0,5.0,3.0,1,...,זכר,תואר ראשון - שנה רביעית,"לא, אין ניסיון",8.0,כן,כן,1,5.0,5.0,4.0
949,1384,A2_289779,Slalom,4,5.0,5.0,9.0,4.0,4.0,1,...,זכר,תואר ראשון - שנה רביעית,"לא, אין ניסיון",8.0,כן,כן,1,5.0,5.0,4.0


In [28]:
display(df2_merge)

,Id,Scenario,require mentally,require physically,required execution rate:,successful performing,required to work hard,Frustration level,Condition,triggered_by,...,Gender,status,experience simulator,drivers license years,experience computer games,accident non-keeping distance,num of times,technologies life easier,understand technology,enjoy understanding technology
0,A1_012594,ACC,7,2.0,7.0,6.0,2.0,1.0,LOAD2_TTC1,Egocar,...,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
1,A1_012594,ACC,6,2.0,5.0,8.0,2.0,1.0,LOAD1_TTC1,Egocar,...,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
2,A1_012594,ACC,7,2.0,8.0,1.0,2.0,1.0,LOAD2_TTC2,Egocar,...,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
3,A1_012594,ACC,7,3.0,6.0,1.0,5.0,3.0,LOAD3_TTC1,Egocar,...,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
4,A1_012594,ACC,8,2.0,6.0,9.0,4.0,2.0,LOAD1_TTC2,Egocar,...,זכר,תואר ראשון - שנה שניה,לא,7.0,לא,לא,0,5.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1381,A5_292076,Slalom,3,5.0,6.0,7.0,4.0,2.0,2,RealVehicle,...,זכר,תואר ראשון - שנה שלישית,"לא, אין ניסיון",6.0,"כן, אבל בקטנה ממש",לא,0,5.0,4.0,4.0
1382,A2_289779,Slalom,7,NaN,6.0,7.0,5.0,5.0,2,RealVehicle,...,זכר,תואר ראשון - שנה רביעית,"לא, אין ניסיון",8.0,כן,כן,1,5.0,5.0,4.0
1383,A2_289779,Slalom,5,5.0,5.0,7.0,5.0,3.0,1,RealVehicle,...,זכר,תואר ראשון - שנה רביעית,"לא, אין ניסיון",8.0,כן,כן,1,5.0,5.0,4.0
1384,A2_289779,Slalom,4,5.0,5.0,9.0,4.0,4.0,1,Egocar,...,זכר,תואר ראשון - שנה רביעית,"לא, אין ניסיון",8.0,כן,כן,1,5.0,5.0,4.0


In [29]:
list(df2_merge.columns)

['Id',
 'Scenario',
 'require mentally',
 'require physically',
 'required execution rate:',
 'successful performing',
 'required to work hard',
 'Frustration level',
 'Condition',
 'triggered_by',
 'Age',
 'Gender',
 'status',
 'experience simulator',
 'drivers license years',
 'experience computer games',
 'accident non-keeping distance',
 'num of times',
 'technologies life easier',
 'understand technology',
 'enjoy understanding technology']

In [47]:
df_Nasa_tlx= df2_merge.groupby(["Scenario",'Condition', 'triggered_by']).agg(
    min_Age=pd.NamedAgg(column="Age", aggfunc="min"),
    max_Age=pd.NamedAgg(column="Age", aggfunc="max"),
    average_Age=pd.NamedAgg(column="Age", aggfunc=np.mean),
    Std_Age=pd.NamedAgg(column="Age", aggfunc=np.std),
   
    average_require_mentally=pd.NamedAgg(column="require mentally", aggfunc=np.mean),
    average_require_physically=pd.NamedAgg(column="require physically", aggfunc=np.mean),
    average_required_execution_rate=pd.NamedAgg(column="required execution rate:", aggfunc=np.mean),
    
    average_successful_performing=pd.NamedAgg(column="successful performing", aggfunc=np.mean),
    average_required_to_work_hard=pd.NamedAgg(column="required to work hard", aggfunc=np.mean),
    average_Frustration_level=pd.NamedAgg(column="Frustration level", aggfunc=np.mean),
    
  
)
display(df_Nasa_tlx)

min_Age  max_Age  average_Age   Std_Age  \
Scenario Condition   triggered_by                                            
ACC      LOAD1_TTC1  Egocar           19.0     30.0    24.374118  1.849527   
         LOAD1_TTC2  Egocar           19.0     27.0    24.185000  1.682005   
         LOAD2_TTC1  Egocar           19.0     27.0    24.294595  1.666831   
         LOAD2_TTC2  Egocar           19.0     28.0    24.266667  1.768018   
         LOAD3_TTC1  Egocar           19.0     32.0    24.354762  1.890586   
         LOAD3_TTC2  Egocar           19.0     32.0    24.362353  1.905754   
Latency  Latency1    Egocar           19.0     30.0    24.383133  1.909720   
         Latency2    Egocar           19.0     32.0    24.338824  1.934898   
         Latency3    Egocar           19.0     28.0    24.314634  1.732487   
Slalom   1           Egocar           24.0     32.0    26.909091  2.385563   
                     RealVehicle      24.0     32.0    26.909091  2.385563   
         2           Egocar           24.0     32.0    26.909091  2.385563   
                     RealVehicle      24.0     32.0    27.000000  2.494438   
Spatial  Allocentric Egocar           19.0     32.0    24.472500  2.004361   
         Egocentric  Egocar           19.0     30.0    24.412987  1.864160   

                                   average_require_mentally  \
Scenario Condition   triggered_by                             
ACC      LOAD1_TTC1  Egocar                        5.197674   
         LOAD1_TTC2  Egocar                        5.148148   
         LOAD2_TTC1  Egocar                        4.773333   
         LOAD2_TTC2  Egocar                        5.050633   
         LOAD3_TTC1  Egocar                        5.376471   
         LOAD3_TTC2  Egocar                        5.244186   
Latency  Latency1    Egocar                        4.845238   
         Latency2    Egocar                        5.104651   
         Latency3    Egocar                        5.313253   
Slalom   1           Egocar                        4.583333   
                     RealVehicle                   5.000000   
         2           Egocar                        4.166667   
                     RealVehicle                   4.454545   
Spatial  Allocentric Egocar                        4.641975   
         Egocentric  Egocar                        4.820513   

                                   average_require_physically  \
Scenario Condition   triggered_by                               
ACC      LOAD1_TTC1  Egocar                          3.709302   
         LOAD1_TTC2  Egocar                          3.753086   
         LOAD2_TTC1  Egocar                          3.240000   
         LOAD2_TTC2  Egocar                          3.367089   
         LOAD3_TTC1  Egocar                          3.722892   
         LOAD3_TTC2  Egocar                          3.465116   
Latency  Latency1    Egocar                          3.976190   
         Latency2    Egocar                          4.011628   
         Latency3    Egocar                          4.325301   
Slalom   1           Egocar                          4.666667   
                     RealVehicle                     5.272727   
         2           Egocar                          5.250000   
                     RealVehicle                     5.200000   
Spatial  Allocentric Egocar                          3.839506   
         Egocentric  Egocar                          3.923077   

                                   average_required_execution_rate  \
Scenario Condition   triggered_by                                    
ACC      LOAD1_TTC1  Egocar                               5.470588   
         LOAD1_TTC2  Egocar                               5.387500   
         LOAD2_TTC1  Egocar                               4.933333   
         LOAD2_TTC2  Egocar                               5.217949   
         LOAD3_TTC1  Egocar                               5.835294   
         LOAD3_TTC2  Egocar             

In [36]:
df2_merge.to_csv('df_merge_final_1.csv')

In [43]:
df2_merge.isna().sum()

Id                                 0
Scenario                           0
require mentally                   0
require physically                 4
required execution rate:           5
successful performing              5
required to work hard              1
Frustration level                  1
Condition                          0
triggered_by                       0
Age                               15
Gender                            15
status                            15
experience simulator               0
drivers license years             15
experience computer games         15
accident non-keeping distance     15
num of times                       0
technologies life easier          15
understand technology             15
enjoy understanding technology    15
dtype: int64

In [36]:
df2_merge.to_csv('df_Nasa_tlxfinal_2.csv')